## Logbook 23 Oct:
- Wrote message at Arduino side and sent to the host, at the host side, the input message looks good, and saved as int list
- Found few problem:
    * sequence number should be binary
    * received checksum and calculated checksum do not match

- Saved the sample pulse message into csv and wrote data processing functions for it (data stored as string, thus add int() to match the functions)

In [2]:
## Remember to use binary for sequence number
print(bin(134|128))

0b10000110


## Code so far

### Read data from a csv file

In [5]:
import csv

class data:
    pass

# Read sample message from the data:
filename = '/Users/yunyao/Desktop/BMET2922/messageExample.csv'
file        = open(filename)
csvreader   = csv.reader(file)

#--------------------Save all the example data into a class--------------
data.raw = []

for row in csvreader:
    data.raw.append(row)
file.close()



### For one of these message:

In [7]:
this_message    = []
lineCnt         = 0
for row in data.raw:

    this_message = row
    lineCnt += 1
    
### This part is supposed to be in the loop, we put it outside to just check for the last message
# Print out this data:
print("Print the input message in Char: ")
for i in range(len(this_message)):
    if i == 1 or i == len(this_message)-3:
        print(this_message[i],end=" ")   
    elif i == 0 or i%4 == 2:
        print(chr(int(this_message[i])),end=" ")
    else: 
        print(chr(int(this_message[i])),end="")
print("\n")

# Print the input message in decimal bytes:
print("Print the input message in decimal bytes: ")
for i in range(len(this_message)):
    print((int(this_message[i])),end=" ")
print("\n")

Print the input message in Char: 
ÿ 134 W 2817 2652 1924 1695 2471 2153 2625 2978 3616 3142 3597 3088 2732 3917 2404 3743 1395 1749 2407 3936 1302 3767 1045 1563 2340 3398 3896 1051 1807 2767 1997 1978 2650 3809 1330 2360 3716 2161 2293 3209 3753 3098 2130 3634 3543 3889 2088 3031 2692 1401 239 ÿ


Print the input message in decimal bytes: 
255 134 87 50 56 49 55 50 54 53 50 49 57 50 52 49 54 57 53 50 52 55 49 50 49 53 51 50 54 50 53 50 57 55 56 51 54 49 54 51 49 52 50 51 53 57 55 51 48 56 56 50 55 51 50 51 57 49 55 50 52 48 52 51 55 52 51 49 51 57 53 49 55 52 57 50 52 48 55 51 57 51 54 49 51 48 50 51 55 54 55 49 48 52 53 49 53 54 51 50 51 52 48 51 51 57 56 51 56 57 54 49 48 53 49 49 56 48 55 50 55 54 55 49 57 57 55 49 57 55 56 50 54 53 48 51 56 48 57 49 51 51 48 50 51 54 48 51 55 49 54 50 49 54 49 50 50 57 51 51 50 48 57 51 55 53 51 51 48 57 56 50 49 51 48 51 54 51 52 51 53 52 51 51 56 56 57 50 48 56 56 51 48 51 49 50 54 57 50 49 52 48 49 239 255 10 



### Checksum

In [8]:
## checksum function: return true if the received checksum and calculated checksum matched
def checksum(this_message):
    # get the checksum of the message
    checksumPo  = len(this_message) - 3
    checksum1 = int(this_message[checksumPo])

    # calculate the checksum
    sum = 0
    for i in range(checksumPo):
        # the int(chr(int(message))) is the corresponding char in decimal bytes 
        # e.g 65 (for 'A'), 49 (for '1)

        sum += int(this_message[i])          

    checksum2 = sum | 128

    print("Add the decimal bytes before checksum: ",sum)
    print("Compare the received checksum and calculated check sum:",checksum1,checksum2)

    if checksum1 == checksum2:
        return True
    else: return False

#-------------- Checksum ------------------------------------
print("Checksume validation: ",checksum(this_message))

Add the decimal bytes before checksum:  10884
Compare the received checksum and calculated check sum: 239 10884
Checksume validation:  False


### Strach data

In [9]:
## This function transfer the 4-digital-chars into numbers, and store the numbers as int/float list into the class
def fourBytesToNum(dataType,this_message):
    dataStartPo = 3

    if dataType == 66:
        i = dataStartPo
        data.thisBPM = int(chr(int(this_message[i]))) * 1000 + int(chr(int(this_message[i+1]))) * 100 + int(chr(int(this_message[i+2])))*10 + int(chr(int(this_message[i+3])))
        # print("This is a BPM message: ")
    
    if dataType == 87:
        # print("This is a Pulse Waveform message:")
        dataSize = 50
        pulse = [0] * dataSize
        for j in range(dataSize):         # position in the pulse array
            i = dataStartPo + j*4             # position in the raw message
            pulse[j] = int(chr(int(this_message[i]))) * 1000 + int(chr(int(this_message[i+1]))) * 100 + int(chr(int(this_message[i+2])))*10 + int(chr(int(this_message[i+3])))
        #     print(pulse[j],end=" ")
        # print(" ")
        data.thispulse = pulse


#------------ Store the pulse data (transfer the "4-digit-char" into a single number)--------------
dataType = int(this_message[2])
fourBytesToNum(dataType,this_message)
print("Print the pulse data stored into class:",data.thispulse)



Print the pulse data stored into class: [2817, 2652, 1924, 1695, 2471, 2153, 2625, 2978, 3616, 3142, 3597, 3088, 2732, 3917, 2404, 3743, 1395, 1749, 2407, 3936, 1302, 3767, 1045, 1563, 2340, 3398, 3896, 1051, 1807, 2767, 1997, 1978, 2650, 3809, 1330, 2360, 3716, 2161, 2293, 3209, 3753, 3098, 2130, 3634, 3543, 3889, 2088, 3031, 2692, 1401]
